### Import and config

In [70]:
# Imports
import os
import time
import random
import json
import logging

from dotenv import load_dotenv
from tqdm import tqdm

import pandas as pd
import requests

from sqlalchemy import create_engine, text, inspect, Text, Integer, BigInteger, Float, Boolean
from sqlalchemy.dialects.postgresql import JSONB

# Configuration
load_dotenv()

# API urls
AUTH_URL = os.getenv('AUTH_URL')
ACTIVITIES_URL = os.getenv('ACTIVITIES_URL')
ACTIVITY_DETAIL_URL_TMPL = os.getenv('ACTIVITY_DETAIL_URL_TMPL')
ACTIVITY_KUDOS_URL_TMPL = os.getenv('ACTIVITY_KUDOS_URL_TMPL')
ACTIVITY_ZONES_URL_TMPL = os.getenv('ACTIVITY_ZONES_URL_TMPL')

# API keys
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

# DB
DB_URI = os.getenv('DB_URI')

# Bronze tables
TARGET_B_SCHEMA = os.getenv('TARGET_B_SCHEMA')
ACTIVITIES_B_TABLE = os.getenv('ACTIVITIES_B_TABLE')
DETAILS_B_TABLE = os.getenv('DETAILS_B_TABLE')
KUDOS_B_TABLE = os.getenv('KUDOS_B_TABLE')
ZONES_B_TABLE = os.getenv('ZONES_B_TABLE')

# Pagination
PER_PAGE = int(os.getenv('PER_PAGE'))
MAX_PAGES = int(os.getenv('MAX_PAGES'))

# Timeouts and retries
REQUEST_TIMEOUT = int(os.getenv('REQUEST_TIMEOUT'))
MAX_RETRIES = int(os.getenv('MAX_RETRIES'))
BASE_SLEEP = float(os.getenv('BASE_SLEEP'))

REFRESH_THRESHOLD_DAYS = int(os.getenv('REFRESH_THRESHOLD_DAYS'))

# Other
LOG_LEVEL = os.getenv('LOG_LEVEL')

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
    format="%(asctime)s | %(levelname)s | %(message)s"
)

pd.set_option('display.max_columns', None)

### API keys validation

In [35]:
REQUIRED_API_ENV = ['CLIENT_ID', 'CLIENT_SECRET', 'REFRESH_TOKEN']
missing_api_env = [env for env in REQUIRED_API_ENV if not os.getenv(env)]
if missing_api_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_api_env)}.")

### DB names validation

In [36]:
REQUIRED_DB_ENV = ['DB_URI', 'TARGET_B_SCHEMA', 'ACTIVITIES_B_TABLE', 'DETAILS_B_TABLE', 'ZONES_B_TABLE']
missing_db_env = [env for env in REQUIRED_DB_ENV if not os.getenv(env)]
if missing_db_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_db_env)}.")

### Connecting with PostgreSQL

In [37]:
engine = create_engine(
  DB_URI, 
  pool_pre_ping=True, 
  pool_size=5, 
  max_overflow=10
)
with engine.connect() as conn:
  conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_B_SCHEMA};"))
logging.info(f"Connection established and {TARGET_B_SCHEMA} schema exists.")

2025-09-29 11:38:10,360 | INFO | Connection established and bronze schema exists.


### Token authorization

In [38]:
def get_access_token(auth_url: str, client_id: str, client_secret: str, refresh_token: str, timeout: int = 30) -> str:
  """
  Request a new access token from an OAuth2 authentication endpoint.

  The function uses a refresh token to obtain a short-lived access token. 
  If the response does not contain an ``access_token``, a RuntimeError is raised.

  Parameters
  ----------
  auth_url : str
      URL of the OAuth2 token endpoint.
  client_id : str
      OAuth2 client identifier.
  client_secret : str
      OAuth2 client secret.
  refresh_token : str
      Refresh token used to request a new access token.
  timeout : int, default=30
      Timeout in seconds for the HTTP request.

  Returns
  -------
  str
      The access token string retrieved from the authentication server.

  Raises
  ------
  RuntimeError
      If the response does not include an ``access_token``.
  requests.exceptions.RequestException
      If the HTTP request fails (e.g., network error, timeout).

  Notes
  -----
  Logs an informational message when the token is successfully retrieved.
  """

  payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': 'refresh_token',
  }
  res = requests.post(auth_url, data=payload, timeout=timeout)
  data = res.json()
  token = data.get('access_token')
  if not token:
    raise RuntimeError(f"No access token in response: {data}")
  logging.info('Access token retrived.')
  return token

access_token = get_access_token(AUTH_URL, CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)

2025-09-29 11:38:11,999 | INFO | Access token retrived.


### HTTP session

In [40]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})

def get_json_with_retry(url: str, params=None, max_retries: int = MAX_RETRIES, timeout: int = REQUEST_TIMEOUT, base_sleep: float = BASE_SLEEP):
  """
  Send a GET request with automatic retries and return the parsed JSON response.

  The function retries on:
    * HTTP 429 (rate limiting) — respects the ``Retry-After`` header if present,
      otherwise waits an increasing backoff time.
    * HTTP 5xx errors — retries with exponential backoff and jitter.
    * Network/connection errors — retries with exponential backoff and jitter.

  For each attempt, the backoff time increases by ``base_sleep * attempt`` seconds
  plus a small random jitter. On the final attempt, any error is raised.

  Parameters
  ----------
  url : str
      Endpoint URL to send the GET request to.
  params : dict, optional
      Query string parameters to include in the request.
  max_retries : int, default=MAX_RETRIES
      Maximum number of retry attempts before failing.
  timeout : int, default=REQUEST_TIMEOUT
      Timeout in seconds for each HTTP request.
  base_sleep : float, default=BASE_SLEEP
      Base number of seconds used for exponential backoff between retries.

  Returns
  -------
  dict
      Parsed JSON response from the server.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after all retry attempts.
  requests.exceptions.HTTPError
      If the server returns a 4xx/5xx response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * HTTP 429 triggers a wait using ``Retry-After`` if available, otherwise a fallback.
  * Logs warnings before each retry and errors if JSON parsing fails.
  * Uses a global ``requests.Session`` (`session`) for connection pooling.
  """

  for attempt in range(1, max_retries + 1):
    try:
      resp = session.get(url, params=params, timeout=timeout)
      
      if resp.status_code == 429:
        retry_after = resp.headers.get('Retry-After')
        if retry_after and retry_after.isdigit():
          sleep_for = int(retry_after)
        else:
          sleep_for = max(base_sleep * attempt, 15)
        logging.warning(f"HTTP 429 - wait {sleep_for}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      if 500 <= resp.status_code < 600:
        if attempt == max_retries:
          resp.raise_for_status()
        sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
        logging.warning(f"HTTP {resp.status_code} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      resp.raise_for_status()

      try:
        return resp.json()
      except json.JSONDecodeError:
        logging.error('JSON parsing error')
        raise
    
    except requests.exceptions.RequestException as e:
      if attempt == max_retries:
        logging.exception('Request error (last attempt)')
        raise
      sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
      logging.warning(f"{e} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
      time.sleep(sleep_for)

### Getting activities list

In [41]:
def fetch_all_activities(activites_url: str, per_page: int = PER_PAGE, max_pages: int = MAX_PAGES):
  """
  Fetch all activity records from a paginated API endpoint.

  The function iterates through API pages until either:
    * the maximum number of pages is reached (``max_pages``), or
    * the API returns an empty list (end of results).

  Each page is retrieved using ``get_json_with_retry`` to ensure resilience
  against transient errors (rate limits, timeouts, 5xx responses).

  Parameters
  ----------
  activites_url : str
      The base URL of the activities endpoint (must support ``per_page`` and ``page`` query params).
  per_page : int, default=PER_PAGE
      Number of activity records to request per page.
  max_pages : int, default=MAX_PAGES
      Maximum number of pages to fetch before stopping.

  Returns
  -------
  list of dict
      Combined list of activity objects returned by the API.

  Raises
  ------
  RuntimeError
      If the API response is not a list (unexpected schema).
  requests.exceptions.RequestException
      If the underlying HTTP requests fail after retries.

  Notes
  -----
  * Logs the number of records downloaded per page and the running total.
  * Stops early if the API returns an empty list before reaching ``max_pages``.
  * The total number of records is ``per_page * n_pages`` at most.
  """

  all_items = []
  page = 1
  while page <= max_pages:
    params = {'per_page': per_page, 'page': page}
    data = get_json_with_retry(activites_url, params=params)

    if not isinstance(data, list):
      raise RuntimeError(f"Unexpected response type for page {page}: {type(data)} — expected list")
    
    if not data:
      break
    
    all_items.extend(data)
    logging.info(f"Page {page}: downloaded {len(data)} records (total: {len(all_items)})")
    page += 1
  return all_items

activities_raw = fetch_all_activities(ACTIVITIES_URL, per_page=PER_PAGE, max_pages=MAX_PAGES)
logging.info(f"Total activities downloaded: {len(activities_raw)}")
activities_df = pd.json_normalize(activities_raw, sep='_')
if activities_df.empty:
    raise RuntimeError('No activities to save.')
activities_df.head()

2025-09-29 11:38:28,463 | INFO | Page 1: downloaded 200 records (total: 200)
2025-09-29 11:38:34,785 | INFO | Page 2: downloaded 200 records (total: 400)
2025-09-29 11:38:41,327 | INFO | Page 3: downloaded 200 records (total: 600)
2025-09-29 11:38:47,533 | INFO | Page 4: downloaded 200 records (total: 800)
2025-09-29 11:38:52,298 | INFO | Page 5: downloaded 200 records (total: 1000)
2025-09-29 11:38:53,224 | INFO | Page 6: downloaded 108 records (total: 1108)
2025-09-29 11:38:53,427 | INFO | Total activities downloaded: 1108


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,...,map_id,map_summary_polyline,map_resource_state,workout_type,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,average_temp
0,2,Afternoon Walk,10054.4,6976,9605,30.0,Walk,Walk,15966567502,2025-09-28T12:32:55Z,...,a15966567502,om|vHatogB`@i@b@e@POZKVBh@dBLJPHt@IHE~@ON@NDT\...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,19km Race Practice Long Run🧪,19064.8,6208,6208,33.0,Run,Run,15963690569,2025-09-28T09:48:45Z,...,a15963690569,{h|vHivogBh@rA@VAZI^aAlBe@p@yAjBiDjFaBvBoB|CwB...,2,2.0,371.6,496.0,374.0,True,2307.3,NaN
2,2,Afternoon Weight Training,0.0,3556,3556,0.0,Workout,WeightTraining,15942185639,2025-09-26T12:01:13Z,...,a15942185639,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Afternoon Ride,11364.9,1730,5936,30.0,Ride,Ride,15942388262,2025-09-26T11:43:37Z,...,a15942388262,mt{vH_aogBNv@t@hCf@bAlA|C~@vCNz@~@~GV~@hEfNTnA...,2,10.0,163.6,NaN,NaN,False,283.0,18.0
4,2,Under and Overs 2km🤺,9171.4,2954,3014,12.0,Run,Run,15933335051,2025-09-25T15:02:42Z,...,a15933335051,yp{vHsxngBlAzCt@|BLj@bAjHLp@Pp@p@xBZx@p@~BP^x@...,2,3.0,383.9,544.0,394.0,True,1135.0,NaN


### Set `bronze.activities` types map

In [42]:
activities_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_summary_polyline": Text,
    "map_resource_state": Integer,
    "average_temp": Float,
}

### Save `bronze.activities` to database

In [43]:
table_full_name = f"{TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"
logging.warning(f"Whole table {table_full_name} will be overwritten.")
activities_df.to_sql(
  ACTIVITIES_B_TABLE, 
  engine, 
  schema=TARGET_B_SCHEMA, 
  if_exists="replace", 
  index=False, 
  method="multi", 
  chunksize=1000, 
  dtype=activities_dtype_map
)

2025-09-29 11:38:58,437 | WARNING | Whole table bronze.activities will be overwritten.


-2

### Identify missing details — only `id` that are not present in `bronze.activities_details`

In [44]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  existing_ids = pd.read_sql(text(f"SELECT id FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)
  existing_ids_set = set(existing_ids['id'].astype('Int64').dropna().to_list())

all_ids_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids = sorted(all_ids_set - existing_ids_set)
logging.info(f"Missing details of {len(missing_ids)} IDs")
pd.DataFrame({'id': missing_ids}).head()

2025-09-29 11:39:04,249 | INFO | Missing details of 1 IDs


,id
0,15191012381


### Identify recent activities — update and download kudos of `id` from the last 30 days

In [45]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  bronze_activities_df = pd.read_sql(text(f"SELECT * FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

In [46]:
bronze_activities_df['date'] =  pd.to_datetime(bronze_activities_df['start_date_local']).dt.tz_localize(None)
today = pd.Timestamp('today').normalize()
bronze_activities_df['activity_age_days'] = (today - bronze_activities_df['date'].dt.normalize()).dt.days

In [47]:
recent_activities_df = bronze_activities_df[bronze_activities_df['activity_age_days'] <= REFRESH_THRESHOLD_DAYS]
recent_activities_ids = set(recent_activities_df['id'].astype('Int64').dropna().to_list())
recent_activities_ids = sorted(recent_activities_ids)
logging.info(f"Refreshing details of {len(recent_activities_ids)} IDs")
pd.DataFrame({'id': recent_activities_ids}).head()

2025-09-29 11:39:14,483 | INFO | Refreshing details of 36 IDs


,id
0,15638783706
1,15645579969
2,15650786884
3,15660079944
4,15683435146


### Download activities details

In [48]:
def fetch_activity_details(activity_id: int):
  """
  Fetch detailed information for a single activity.

  Builds the activity detail endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity details as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_DETAIL_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_DETAIL_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

ids_to_be_downloaded = sorted(set(missing_ids) | set(recent_activities_ids))
details_records = []
for i, act_id in tqdm(enumerate(ids_to_be_downloaded, start=1), total=len(ids_to_be_downloaded)):
  try:
    resp = fetch_activity_details(act_id)
    
    if not isinstance(resp, dict):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue
    details_records.append(resp)
    
  except Exception as e:
    logging.error(f"Error downloading details for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Details downloaded: {len(details_records)} / {len(ids_to_be_downloaded)}")
details_df_new = pd.json_normalize(details_records, sep='_')

100%|██████████| 37/37 [05:24<00:00,  8.78s/it]
2025-09-29 11:44:58,751 | INFO | Details downloaded: 37 / 37


In [49]:
if not details_df_new.empty:
  details_df_new = details_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  details_df_new = details_df_new.dropna(how='all')
  
details_df_new.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,similar_activities_trend_speeds,similar_activities_trend_current_activity_index,similar_activities_trend_min_speed,similar_activities_trend_mid_speed,similar_activities_trend_max_speed,similar_activities_trend_direction,similar_activities_resource_state
0,3,Deepspot🤿,0.0,3600,3600,0.0,Workout,Workout,NaN,15191012381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Coffee Race 2025☕️,46598.1,10100,17355,126.0,Ride,Ride,11.0,15638783706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,24km Race Practice Long Run🪦,8565.3,2985,3081,15.0,Run,Run,2.0,15645579969,...,2.869447,NaN,NaN,[2.8694472361809042],0.0,2.869447,2.869447,2.869447,0.0,2.0
3,3,Afternoon Ride,24309.7,5770,7708,63.0,Ride,Ride,10.0,15650786884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,Afternoon Walk,5062.4,3664,3840,12.0,Walk,Walk,NaN,15660079944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Set `bronze.activities_details` types map

In [50]:
activities_details_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "perceived_exertion": Text,
    "prefer_perceived_exertion": Text,
    "segment_efforts": JSONB,
    "splits_metric": JSONB,
    "splits_standard": JSONB,
    "laps": JSONB,
    "best_efforts": JSONB,
    "stats_visibility": JSONB,
    "hide_from_home": Boolean,
    "device_name": Text,
    "embed_token": Text,
    "available_zones": JSONB,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "map_summary_polyline": Text,
    "gear_primary": Boolean,
    "gear_name": Text,
    "gear_nickname": Text,
    "gear_resource_state": Float,
    "gear_retired": Boolean,
    "gear_distance": Float,
    "gear_converted_distance": Float,
    "photos_primary": JSONB,
    "photos_count": Integer,
    "similar_activities_effort_count": Float,
    "similar_activities_average_speed": Float,
    "similar_activities_min_average_speed": Float,
    "similar_activities_mid_average_speed": Float,
    "similar_activities_max_average_speed": Float,
    "similar_activities_pr_rank": Float,
    "similar_activities_frequency_milestone": Float,
    "similar_activities_trend_speeds": JSONB,
    "similar_activities_trend_current_activity_index": Float,
    "similar_activities_trend_min_speed": Float,
    "similar_activities_trend_mid_speed": Float,
    "similar_activities_trend_max_speed": Float,
    "similar_activities_trend_direction": Float,
    "similar_activities_resource_state": Float,
    "average_temp": Float,
    "photos_primary_unique_id": Text,
    "photos_primary_urls_600": Text,
    "photos_primary_urls_100": Text,
    "photos_primary_source": Integer,
    "photos_primary_media_type": Integer,
    "photos_use_primary_photo": Boolean,
    "private_note": Text
}

### Save `bronze.activities_details` to database

In [51]:
if details_df_new.empty:
  logging.info('No new details to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA):
    details_df_new.head(0).to_sql(
      DETAILS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_details_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE details_stg
        AS TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    details_df_new.to_sql(
      "details_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_details_dtype_map
      )
    
    # Prepare upsert
    stg_cols = details_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in details_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM details_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS details_stg;'))
  
    logging.info('Activities details saved to PostgreSQL.')

2025-09-29 11:47:32,555 | INFO | Activities details saved to PostgreSQL.


### Sanity check

In [52]:
with engine.begin() as conn:
    total_acts = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"), conn)
    total_det = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}: {int(total_acts['n'][0])}")
logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}: {int(total_det['n'][0])}")

2025-09-29 11:47:35,347 | INFO | Total number of records in bronze.activities: 1108
2025-09-29 11:47:35,348 | INFO | Total number of records in bronze.activities_details: 1108


### Identify missing kudos — only `activity_id` that are not present in `bronze.kudos`

In [62]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} (id TEXT PRIMARY KEY, activity_id BIGINT)
"""))
  existing_ids_kudos = pd.read_sql(text(f"SELECT activity_id FROM {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}"), conn)
  existing_ids_kudos_set = set(existing_ids_kudos['activity_id'].astype('Int64').dropna().to_list())

all_ids_kudos_set = set(activities_df[activities_df['kudos_count'] > 0]['id'].astype('Int64').dropna().to_list())
missing_ids_kudos = sorted(all_ids_kudos_set - existing_ids_kudos_set, reverse=True)
logging.info(f"Missing kudos of {len(missing_ids_kudos)} activities")
pd.DataFrame({'activity_id': missing_ids_kudos}).head()

2025-09-29 11:51:10,000 | INFO | Missing kudos of 0 activities


,activity_id


### Download activities kudos

In [63]:
def fetch_activity_kudos(activity_id: int):
  """
  Fetch all kudos for a single activity.

  Builds the activity kudos endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity kudos as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_KUDOS_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_KUDOS_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

kudos_to_be_downloaded = sorted(set(missing_ids_kudos) | set(recent_activities_ids))

kudos_records = []
for i, act_id in tqdm(enumerate(kudos_to_be_downloaded, start=1), total=len(kudos_to_be_downloaded)):
  try:
    resp = fetch_activity_kudos(act_id)
    
    if not isinstance(resp, list):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue

    for i, kudos in enumerate(resp):
      kudos["activity_id"] = act_id
      kudos["kudos_id"] = i
    kudos_records.extend(resp)
    
  except Exception as e:
    logging.error(f"Error downloading kudos for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Kudos downloaded: {len(kudos_records)} / {len(kudos_to_be_downloaded)}")


100%|██████████| 36/36 [04:53<00:00,  8.16s/it]
2025-09-29 11:56:11,790 | INFO | Kudos downloaded: 362 / 36


In [64]:

kudos_df_new = pd.json_normalize(kudos_records, sep='_')
if not kudos_df_new.empty:
  kudos_df_new = kudos_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  kudos_df_new = kudos_df_new.dropna(how='all')
  
kudos_df_new['id'] = kudos_df_new[['activity_id', 'kudos_id']].astype("string").agg("-".join, axis=1)

In [65]:
kudos_df_new.head()

,resource_state,firstname,lastname,activity_id,kudos_id,id
0,2,Jakub,Ł.,15638783706,0,15638783706-0
1,2,Paula,P.,15638783706,1,15638783706-1
2,2,Kacper,G.,15638783706,2,15638783706-2
3,2,Agnieszka,G.,15638783706,3,15638783706-3
4,2,Jacek,S.,15638783706,4,15638783706-4


### Set `bronze.kudos` types map

In [66]:
activities_kudos_dtype_map = {
    "resource_state": Integer,
    "first_name": Text,
    "last_name": Float,
    "activity_id": BigInteger,
    "id": Text
}

### Save `bronze.kudos` to database

In [67]:
if kudos_df_new.empty:
  logging.info('No new kudos to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=KUDOS_B_TABLE, schema=TARGET_B_SCHEMA):
    kudos_df_new.head(0).to_sql(
      KUDOS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_kudos_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=KUDOS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE kudos_stg
        AS TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    kudos_df_new.to_sql(
      "kudos_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_kudos_dtype_map
      )
    
    # Prepare upsert
    stg_cols = kudos_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in kudos_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM kudos_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS kudos_stg;'))
  
    logging.info('Activities kudos saved to PostgreSQL.')

2025-09-29 11:56:24,878 | INFO | Activities kudos saved to PostgreSQL.


### Identify missing zones — only `activity_id` that are not present in `bronze.activities_zones`

In [71]:
activities_df

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_cadence,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete_id,athlete_resource_state,map_id,map_summary_polyline,map_resource_state,workout_type,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,average_temp
0,2,Afternoon Walk,10054.4,6976,9605,30.0,Walk,Walk,15966567502,2025-09-28T12:32:55Z,2025-09-28T14:32:55Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,0,13,0,2,0,False,False,False,False,followers_only,False,None,"[51.107311, 17.124105]","[51.108151, 17.12518]",1.441,2.671,53.3,True,93.1,116.0,False,True,125.6,116.0,1.704782e+10,17047817584,garmin_ping_485149670263,False,0,2,False,14.0,81055898,1,a15966567502,om|vHatogB`@i@b@e@POZKVBh@dBLJPHt@IHE~@ON@NDT\...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,19km Race Practice Long Run🧪,19064.8,6208,6208,33.0,Run,Run,15963690569,2025-09-28T09:48:45Z,2025-09-28T11:48:45Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,9,12,2,1,0,False,False,False,False,everyone,False,g23642256,"[51.107335, 17.124156]","[51.107395, 17.124479]",3.071,4.080,85.4,True,154.5,170.0,False,True,125.0,115.8,1.704488e+10,17044880966,garmin_ping_485086843866,False,5,0,False,144.0,81055898,1,a15963690569,{h|vHivogBh@rA@VAZI^aAlBe@p@yAjBiDjFaBvBoB|CwB...,2,2.0,371.6,496.0,374.0,True,2307.3,NaN
2,2,Afternoon Weight Training,0.0,3556,3556,0.0,Workout,WeightTraining,15942185639,2025-09-26T12:01:13Z,2025-09-26T14:01:13Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,5,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.000,0.000,NaN,True,109.8,154.0,False,True,0.0,0.0,1.702286e+10,17022857086,garmin_ping_484418759467,False,0,0,False,10.0,81055898,1,a15942185639,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Afternoon Ride,11364.9,1730,5936,30.0,Ride,Ride,15942388262,2025-09-26T11:43:37Z,2025-09-26T13:43:37Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,3,7,0,1,0,False,True,False,False,followers_only,False,b12572672,"[51.107515, 17.124195]","[51.10762, 17.124164]",6.569,10.640,NaN,True,129.9,154.0,False,True,129.2,117.8,1.702306e+10,17023064153,garmin_ping_484424920139,False,0,0,False,10.0,81055898,1,a15942388262,mt{vH_aogBNv@t@hCf@bAlA|C~@vCNz@~@~GV~@hEfNTnA...,2,10.0,163.6,NaN,NaN,False,283.0,18.0
4,2,Under and Overs 2km🤺,9171.4,2954,3014,12.0,Run,Run,15933335051,2025-09-25T15:02:42Z,2025-09-25T17:02:42Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,2,9,0,1,0,False,False,False,False,everyone,False,g24134620,"[51.106916, 17.123259]","[51.107875, 17.123719]",3.105,4.040,85.6,True,153.7,174.0,False,True,122.4,114.2,1.701381e+10,17013809239,garmin_ping_484116398751,False,0,0,False,59.0,81055898,1,a15933335051,yp{vHsxngBlAzCt@|BLj@bAjHLp@Pp@p@xBZx@p@~BP^x@...,2,3.0,383.9,544.0,394.0,True,1135.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2,Morning Run,5640.4,2086,2100,44.8,Run,Run,6012647108,2016-07-19T08:07:29Z,2016-07-19T10:07:29Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,1,0,0,1,0,False,False,False,False,followers_only,False,g9239745,"[52.758578, 15.235688]","[52.758743, 15.235118]",2.704,3.700,NaN,False,NaN,NaN,False,False,60.9,38.1,6.390576e+09,6390575750,2016-07-19-100729.gpx,False,1,0,False,NaN,81055898,1,a6012647108,al_aI_v~{AA\e@tA]t@]j@aAbC]l@s@fBUr@]n@QNOVi@d...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1104,2,Morning Run,5067.5,1859,1859,36.7,Run,Ru

In [68]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{ZONES_B_TABLE} (id TEXT PRIMARY KEY, activity_id BIGINT)
"""))
  existing_ids_zones = pd.read_sql(text(f"SELECT activity_id FROM {TARGET_B_SCHEMA}.{ZONES_B_TABLE}"), conn)
  existing_ids_zones_set = set(existing_ids_zones['activity_id'].astype('Int64').dropna().to_list())

all_ids_zones_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids_zones = sorted(all_ids_zones_set - existing_ids_zones_set, reverse=True)
logging.info(f"Missing zones of {len(missing_ids_zones)} activities")
pd.DataFrame({'activity_id': missing_ids_zones}).head()

2025-09-29 11:56:27,075 | INFO | Missing zones of 155 activities


,activity_id
0,15191012381
1,14223271112
2,14164947706
3,11177712211
4,10678868170


### Download activities zones

In [89]:
def fetch_activity_zones(activity_id: int):
  """
  Fetch all zones for a single activity.

  Builds the activity zones endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity zones as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_ZONES_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_ZONES_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)


zones_records = []
for i, act_id in tqdm(enumerate(missing_ids_zones, start=1), total=len(missing_ids_zones)):
  try:
    resp = fetch_activity_zones(act_id)
    
    if not isinstance(resp, list):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue

    if not resp:
      resp = [{'score': None,
              'distribution_buckets': None,
              'type': 'dummy',
              'resource_state': None,
              'sensor_based': None,
              'points': None,
              'custom_zones': None}]

    for i, zones in enumerate(resp):
      zones["activity_id"] = act_id
    zones_records.extend(resp)
    
  except Exception as e:
    logging.error(f"Error downloading zones for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"zones downloaded: {len(zones_records)} / {len(missing_ids_zones)}")

 85%|████████▍ | 131/155 [18:46<03:05,  7.71s/it]2025-09-29 12:29:12,350 | WARNING | HTTP 429 - wait 15s (attempt 1/5)
2025-09-29 12:29:27,834 | WARNING | HTTP 429 - wait 16.0s (attempt 2/5)
2025-09-29 12:29:44,104 | WARNING | HTTP 429 - wait 24.0s (attempt 3/5)
100%|██████████| 155/155 [23:01<00:00,  8.91s/it]
2025-09-29 12:33:26,735 | INFO | zones downloaded: 155 / 155


In [80]:
resp = fetch_activity_zones(15191012381)

In [86]:
resp_test = [{'score': None,
              'distribution_buckets': None,
              'type': None,
              'resource_state': None,
              'sensor_based': None,
              'points': None,
              'custom_zones': None}]

In [27]:
zones_df_new = pd.json_normalize(zones_records, sep='_')
if not zones_df_new.empty:
  zones_df_new = zones_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  zones_df_new = zones_df_new.dropna(how='all')
  
zones_df_new['id'] = zones_df_new[['activity_id', 'type']].astype("string").agg("-".join, axis=1)

In [28]:
zones_df_new

,score,distribution_buckets,type,resource_state,sensor_based,points,custom_zones,activity_id,id
0,14.0,"[{'min': 0, 'max': 133, 'time': 6976.0}, {'min...",heartrate,3,True,0.0,True,15966567502,15966567502-heartrate
1,144.0,"[{'min': 0, 'max': 133, 'time': 27.0}, {'min':...",heartrate,3,True,44.0,True,15963690569,15963690569-heartrate
2,10.0,"[{'max': 2.542, 'min': 0, 'time': 0.0}, {'max'...",pace,3,True,NaN,NaN,15963690569,15963690569-pace
3,NaN,"[{'max': 0, 'min': 0, 'time': 0.0}, {'max': 50...",power,3,True,NaN,NaN,15963690569,15963690569-power
4,10.0,"[{'min': 0, 'max': 133, 'time': 1169.0}, {'min...",heartrate,3,True,0.0,True,15942388262,15942388262-heartrate
5,NaN,"[{'max': 0, 'min': 0, 'time': 224.0}, {'max': ...",power,3,False,NaN,NaN,15942388262,15942388262-power
6,10.0,"[{'min': 0, 'max': 133, 'time': 3334.0}, {'min...",heartrate,3,True,0.0,True,15942185639,15942185639-heartrate
7,59.0,"[{'min': 0, 'max': 133, 'time': 95.0}, {'min':...",heartrate,3,True,25.0,True,15933335051,15933335051-heartrate
8,8.0,"[{'max': 2.542, 'min': 0, 'time': 199.0}, {'ma...",pace,3,True,NaN,NaN,15933335051,15933335051-pace
9,NaN,"[{'max': 0, 'min': 0, 'time': 0.0}, {'max': 50...",power,3,True,NaN,NaN,15933335051,15933335051-power


In [29]:
zones_records

[{'score': 14.0,
  'distribution_buckets': [{'min': 0, 'max': 133, 'time': 6976.0},
   {'min': 134, 'max': 147, 'time': 0.0},
   {'min': 148, 'max': 160, 'time': 0.0},
   {'min': 161, 'max': 166, 'time': 0.0},
   {'min': 167, 'max': -1, 'time': 0.0}],
  'type': 'heartrate',
  'resource_state': 3,
  'sensor_based': True,
  'points': 0,
  'custom_zones': True,
  'activity_id': 15966567502},
 {'score': 144.0,
  'distribution_buckets': [{'min': 0, 'max': 133, 'time': 27.0},
   {'min': 134, 'max': 147, 'time': 1259.0},
   {'min': 148, 'max': 160, 'time': 3024.0},
   {'min': 161, 'max': 166, 'time': 519.0},
   {'min': 167, 'max': -1, 'time': 1379.0}],
  'type': 'heartrate',
  'resource_state': 3,
  'sensor_based': True,
  'points': 44,
  'custom_zones': True,
  'activity_id': 15963690569},
 {'score': 10,
  'distribution_buckets': [{'max': 2.542, 'min': 0, 'time': 0.0},
   {'max': 2.953, 'min': 2.542, 'time': 2526.0},
   {'max': 3.289, 'min': 2.953, 'time': 1805.0},
   {'max': 3.514, 'min': 3

In [30]:

zones_df_new = pd.json_normalize(zones_records, sep='_')
if not zones_df_new.empty:
  zones_df_new = zones_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  zones_df_new = zones_df_new.dropna(how='all')
  
zones_df_new['id'] = zones_df_new[['activity_id', 'type']].astype("string").agg("-".join, axis=1)

In [31]:
zones_df_new.head()

,score,distribution_buckets,type,resource_state,sensor_based,points,custom_zones,activity_id,id
0,14.0,"[{'min': 0, 'max': 133, 'time': 6976.0}, {'min...",heartrate,3,True,0.0,True,15966567502,15966567502-heartrate
1,144.0,"[{'min': 0, 'max': 133, 'time': 27.0}, {'min':...",heartrate,3,True,44.0,True,15963690569,15963690569-heartrate
2,10.0,"[{'max': 2.542, 'min': 0, 'time': 0.0}, {'max'...",pace,3,True,NaN,NaN,15963690569,15963690569-pace
3,NaN,"[{'max': 0, 'min': 0, 'time': 0.0}, {'max': 50...",power,3,True,NaN,NaN,15963690569,15963690569-power
4,10.0,"[{'min': 0, 'max': 133, 'time': 1169.0}, {'min...",heartrate,3,True,0.0,True,15942388262,15942388262-heartrate


### Set `bronze.activities_zones` types map

In [32]:
activities_zones_dtype_map = {
    "score": Float,
    "distribution_buckets": JSONB,
    "type": Text,
    "resource_state": Text,
    "sensor_based": Boolean,
    "points": Float,
    "custom_zones": Boolean,
    "activity_id": BigInteger,
    "id": Text
}

### Save `bronze.activities_zones` to database

In [33]:
if zones_df_new.empty:
  logging.info('No new kudos to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=ZONES_B_TABLE, schema=TARGET_B_SCHEMA):
    zones_df_new.head(0).to_sql(
      ZONES_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_zones_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=ZONES_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE zones_stg
        AS TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    zones_df_new.to_sql(
      "zones_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_zones_dtype_map
      )
    
    # Prepare upsert
    stg_cols = zones_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in zones_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{ZONES_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM zones_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS zones_stg;'))
  
    logging.info('Activities kudos saved to PostgreSQL.')

2025-09-29 09:05:38,173 | INFO | Activities kudos saved to PostgreSQL.
